In [17]:
import numpy as np
import pandas as pd
# pd.set_option('display.max_rows', None)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from torch.utils.data import Dataset, DataLoader
import torch

import sys
import os
sys.path.append(os.path.abspath(".."))
from utils import preprocess

In [18]:
transaction_df = pd.read_csv('../../데이터/Transaction/transaction_all.csv')
economy_df = pd.read_excel('../../데이터/Economy/economy_all.xlsx')

transaction_df, economy_df = preprocess(transaction_df, economy_df, window_size=5)

In [19]:
df = transaction_df
window_size = 5

### LSTM_Transaction_Train_Dataset

In [5]:
def price_interpolate(group, start, end):
    idx = pd.date_range(start, end, freq='MS')
    group = group.set_index('계약년월').reindex(idx)
    group['동'] = group['동'].fillna(method='ffill')
    group['단지'] = group['단지'].fillna(method='ffill')
    group['건축년도'] = group['건축년도'].fillna(method='ffill')
    group['제곱미터당 거래금액'] = group[['제곱미터당 거래금액']].interpolate()
    # imputer = IterativeImputer(max_iter=10, random_state=0)
    # group['제곱미터당 거래금액'] = imputer.fit_transform(group[['제곱미터당 거래금액']])
    
    return group

In [13]:
class LSTM_Transaction_Train_Dataset(Dataset):
    def __init__(self, df, window_size=5):
        df['계약년월'] = pd.to_datetime(df['계약년월'].astype(str), format='%Y%m')
        interpolated_df = pd.DataFrame(df.groupby(['동', '단지']).apply(lambda group: price_interpolate(group,group['계약년월'].min(),group['계약년월'].max()))['제곱미터당 거래금액']).reset_index().rename(columns={'level_2':'계약년월'})

        dongs_x, dongs_y = [], []
        for dong in interpolated_df['동'].unique():
            for apartment_complex in interpolated_df[interpolated_df['동'] == dong]['단지'].unique():
                filtered_interpolated_df = interpolated_df[(interpolated_df['동'] == dong)*(interpolated_df['단지'] == apartment_complex)]
                filtered_interpolated_df_values = filtered_interpolated_df['제곱미터당 거래금액'].values
                filtered_interpolated_df_times = filtered_interpolated_df['계약년월'].apply(lambda x: float((x.year-pd.Timestamp('2006-01').year)*12+(x.month-pd.Timestamp('2006-01').month)+1)).values
                for idx in range(len(filtered_interpolated_df_values)-window_size):
                    dongs_x.append([filtered_interpolated_df_values[idx:idx+window_size],filtered_interpolated_df_times[idx:idx+window_size]])
                    dongs_y.append([filtered_interpolated_df_values[idx+window_size:idx+window_size+1],filtered_interpolated_df_times[idx+window_size:idx+window_size+1]])
        self.dongs_x = dongs_x
        self.dongs_y = dongs_y
        self.len = len(dongs_x)

    # 부동산_x, 부동산_시점_x, 부동산_y, 부동산_시점_y 
    def __getitem__(self, i):
        return torch.FloatTensor(self.dongs_x[i][0]), torch.FloatTensor(self.dongs_x[i][1]), torch.FloatTensor(self.dongs_y[i][0]), torch.FloatTensor(self.dongs_y[i][1])

    def __len__(self):
        return self.len

batch_size = 1
train_dataset = LSTM_Transaction_Train_Dataset(transaction_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [14]:
for x,y,z,w in train_loader:
    print(x)
    print(y)
    print(z)
    print(w)
    break

tensor([[ 7.6849,  8.6174, 10.3881,  7.7170, 10.3881]])
tensor([[3., 4., 5., 6., 7.]])
tensor([[10.3881]])
tensor([[8.]])


### LSTM_Transaction_Val_Dataset

In [20]:
def price_interpolate(group, start, end):
    idx = pd.date_range(start, end, freq='MS')
    group = group.set_index('계약년월').reindex(idx)
    group['동'] = group['동'].fillna(method='ffill')
    group['단지'] = group['단지'].fillna(method='ffill')
    group['건축년도'] = group['건축년도'].fillna(method='ffill')
    group['제곱미터당 거래금액'] = group[['제곱미터당 거래금액']].interpolate()
    # imputer = IterativeImputer(max_iter=10, random_state=0)
    # group['제곱미터당 거래금액'] = imputer.fit_transform(group[['제곱미터당 거래금액']])
    
    return group

In [21]:
class LSTM_Transaction_Val_Dataset(Dataset):
    def __init__(self, df, window_size=5):
        df['계약년월'] = pd.to_datetime(df['계약년월'].astype(str), format='%Y%m')
        df_2022 = df[df['계약년월'].dt.year == 2022]
        df_else = df[df['계약년월'].dt.year != 2022]

        dongs_x, dongs_y = [], []
        for dong in df_2022['동'].unique():
            for apartment_complex in df_2022[df_2022['동'] == dong]['단지'].unique():
                # 2022년 이전 거래 window_size만큼 가져와 2022년 첫 거래 예측
                filtered_df_2022 = df_2022[(df_2022['동'] == dong)*(df_2022['단지'] == apartment_complex)]
                filtered_df_2022_values = filtered_df_2022['제곱미터당 거래금액'].values
                filtered_df_2022_times = filtered_df_2022['계약년월'].apply(lambda x: float((x.year-pd.Timestamp('2006-01').year)*12+(x.month-pd.Timestamp('2006-01').month)+1)).values

                filtered_df_else = df_else[(df_else['동'] == dong)*(df_else['단지'] == apartment_complex)]
                past_date_range = pd.date_range(start='2006-01-01', end=filtered_df_else.iloc[-1]['계약년월']-pd.DateOffset(months=window_size), freq='MS')
                interpolate_date_range = pd.date_range(start=filtered_df_else[filtered_df_else['계약년월'].isin(past_date_range)].iloc[-1]['계약년월'], end=filtered_df_else.iloc[-1]['계약년월'], freq='MS')
                
                double_filtered_df_else = price_interpolate(filtered_df_else[filtered_df_else['계약년월'].isin(interpolate_date_range)],interpolate_date_range[0],interpolate_date_range[-1]).reset_index().rename(columns={'index': '계약년월'})[-window_size:]
                double_filtered_df_else_values = double_filtered_df_else['제곱미터당 거래금액'].values
                double_filtered_df_times = double_filtered_df_else['계약년월'].apply(lambda x: float((x.year-pd.Timestamp('2006-01').year)*12+(x.month-pd.Timestamp('2006-01').month)+1)).values
                dongs_x.append([double_filtered_df_else_values,double_filtered_df_times])
                dongs_y.append([filtered_df_2022_values[0:1],filtered_df_2022_times[0:1]])
                
                # 2022년 나머지 거래 예측
                if len(filtered_df_2022) > 1:
                    filtered_df = df[(df['동'] == dong)*(df['단지'] == apartment_complex)]
                    for idx in range(len(filtered_df_2022)-1):
                        past_date_range = pd.date_range(start='2006-01-01', end=filtered_df_2022.iloc[idx]['계약년월']-pd.DateOffset(months=window_size), freq='MS')
                        interpolate_date_range = pd.date_range(start=filtered_df[filtered_df['계약년월'].isin(past_date_range)].iloc[-1]['계약년월'], end=filtered_df_2022.iloc[idx]['계약년월'], freq='MS')
                        double_filtered_df = price_interpolate(filtered_df[filtered_df['계약년월'].isin(interpolate_date_range)],interpolate_date_range[0],interpolate_date_range[-1]).reset_index().rename(columns={'index': '계약년월'})[-window_size:]
                        double_filtered_df_values = double_filtered_df['제곱미터당 거래금액'].values
                        double_filtered_df_times = double_filtered_df['계약년월'].apply(lambda x: float((x.year-pd.Timestamp('2006-01').year)*12+(x.month-pd.Timestamp('2006-01').month)+1)).values
                        dongs_x.append([double_filtered_df_values,double_filtered_df_times])
                        dongs_y.append([filtered_df_2022_values[idx+1:idx+2],filtered_df_2022_times[idx+1:idx+2]])
                else:
                    continue

        self.dongs_x = dongs_x
        self.dongs_y = dongs_y
        self.len = len(dongs_x)

    # 부동산_x, 부동산_시점_x, 부동산_y, 부동산_시점_y 
    def __getitem__(self, i):
        return torch.FloatTensor(self.dongs_x[i][0]), torch.FloatTensor(self.dongs_x[i][1]), torch.FloatTensor(self.dongs_y[i][0]), torch.FloatTensor(self.dongs_y[i][1])

    def __len__(self):
        return self.len

batch_size = 1
val_dataset = LSTM_Transaction_Val_Dataset(transaction_df)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [22]:
for x,y,z,w in val_loader:
    print(x)
    print(y)
    print(z)
    print(w)
    break

tensor([[25.0094, 27.5103, 27.3019, 27.0935, 26.8851]])
tensor([[188., 189., 190., 191., 192.]])
tensor([[27.5103]])
tensor([[196.]])


### LSTM_Transformer_Dataset

In [7]:
def price_fill_0(df):
    months = pd.to_datetime(pd.date_range(start="20060101", end="20221201", freq='MS'))
    complex_city_combinations = df[['단지', '동']].drop_duplicates()

    combinations = pd.DataFrame({
        '단지': np.tile(complex_city_combinations['단지'], len(months)),
        '동': np.tile(complex_city_combinations['동'], len(months)),
        '계약년월': np.repeat(months, len(complex_city_combinations))
    })
    
    df['계약년월'] = pd.to_datetime(df['계약년월'].astype(str), format='%Y%m')
    df = pd.merge(combinations, df, on=['단지', '계약년월', '동'], how='left')
    df['제곱미터당 거래금액(만원)'].fillna(0, inplace=True)

    return df

In [8]:
class LSTM_Transformer_Dataset(Dataset):
    def __init__(self, transaction_data, economy_data, window_size=5):
        all_dong_max_apartment_complex = 311 # transaction_data.drop_duplicates(subset=['동','단지']).groupby(['동'])['단지'].count().max()

        filled_data = price_fill_0(transaction_data)
        filled_data = filled_data[['동', '단지', '계약년월', '제곱미터당 거래금액']]        

        dongs_x, dongs_y = [], []
        for dong in filled_data['동'].unique():
            filtered_filled_data = filled_data[filled_data['동'] == dong]
            date_range = pd.date_range('20060101', '20221201', freq='MS')
            for idx in range(len(date_range)-window_size):
                current_range_x = date_range[idx:idx+window_size]
                current_range_y = date_range[idx+window_size:idx+window_size+1]
                current_range_filled_x = filtered_filled_data[filtered_filled_data['계약년월'].isin(current_range_x)]
                current_range_filled_y = filtered_filled_data[filtered_filled_data['계약년월'].isin(current_range_y)]
                grouped_current_range_filled_x = current_range_filled_x.groupby('단지').agg({'제곱미터당 거래금액': list}).reset_index()['제곱미터당 거래금액'].to_list()
                grouped_current_range_filled_y = current_range_filled_y.groupby('단지').agg({'제곱미터당 거래금액': list}).reset_index()['제곱미터당 거래금액'].to_list()
                if len(grouped_current_range_filled_x) < all_dong_max_apartment_complex:
                    for _ in range(all_dong_max_apartment_complex-len(grouped_current_range_filled_x)):
                        grouped_current_range_filled_x.append([0.0]*window_size)
                        grouped_current_range_filled_y.append([0.0])
                economy_x, economy_y = [], []
                economy_x.append(economy_data['통화량'][idx:idx+window_size].to_list())
                economy_y.append(economy_data['통화량'][idx+window_size:idx+window_size+1].to_list())
                grouped_current_range_filled_and_economy_x = []
                grouped_current_range_filled_and_economy_x.extend([grouped_current_range_filled_x, economy_x])
                grouped_current_range_filled_and_economy_y = []
                grouped_current_range_filled_and_economy_y.extend([grouped_current_range_filled_y, economy_y])
                dongs_x.append(grouped_current_range_filled_and_economy_x)
                dongs_y.append(grouped_current_range_filled_and_economy_y)

        self.dongs_x = dongs_x
        self.dongs_y = dongs_y
        self.len = len(dongs_x)

    # 부동산_x, 경제_x, 부동산_y, 경제_y 
    def __getitem__(self, i):
        return torch.FloatTensor(self.dongs_x[i][0]), torch.FloatTensor(self.dongs_x[i][1]), torch.FloatTensor(self.dongs_y[i][0]), torch.FloatTensor(self.dongs_y[i][1])

    def __len__(self):
        return self.len

batch_size = 1
train_dataset = LSTM_Transformer_Dataset(transaction_all, economy_all)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [9]:
for x,y,z,w in train_loader:
    print(w.dtype)
    break

torch.float32


### ODE_Transaction_Dataset

In [11]:
class ODE_Transaction_Dataset(Dataset):
    def __init__(self, data, window_size=5):
        data['계약년월'] = pd.to_datetime(data['계약년월'].astype(str), format='%Y%m')
        
        dongs_x, dongs_y = [], []
        for dong in data['동'].unique():
            for apartment_complex in data[data['동'] == dong]['단지'].unique():
                    filtered_data = data[(data['동'] == dong)*(data['단지'] == apartment_complex)]
                    filtered_data_values = filtered_data['제곱미터당 거래금액'].values
                    filtered_data_times = filtered_data['계약년월'].apply(lambda x: float((x.year-pd.Timestamp('2006-01').year)*12+(x.month-pd.Timestamp('2006-01').month)+1)).values
                    for idx in range(len(filtered_data_values)-window_size):
                        dongs_x.append([filtered_data_values[idx:idx+window_size],filtered_data_times[idx:idx+window_size]])
                        dongs_y.append([filtered_data_values[idx+window_size:idx+window_size+1],filtered_data_times[idx+window_size:idx+window_size+1]])
                        
        self.dongs_x = dongs_x
        self.dongs_y = dongs_y
        self.len = len(dongs_x)

    # 부동산_x, 부동산_시점_x, 부동산_y, 부동산_시점_y 
    def __getitem__(self, i):
        return torch.FloatTensor(self.dongs_x[i][0]), torch.FloatTensor(self.dongs_x[i][1]), torch.FloatTensor(self.dongs_y[i][0]), torch.FloatTensor(self.dongs_y[i][1])

    def __len__(self):
        return self.len

batch_size = 1
train_dataset = ODE_Transaction_Dataset(transaction_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [12]:
for x,y,z,w in train_loader:
    print(w.dtype)
    break

torch.float32


### ODE_Transformer_Dataset

In [14]:
class ODE_Transformer_Dataset(Dataset):
    def __init__(self, transaction_data, economy_data, window_size=5):
        all_dong_max_apartment_complex = 311 # transaction_data.drop_duplicates(subset=['동','단지']).groupby(['동'])['단지'].count().max()

        transaction_data['계약년월'] = pd.to_datetime(transaction_data['계약년월'].astype(str), format='%Y%m')
        date_range = pd.date_range('20060101', '20221201', freq='MS')
        economy_data.index = date_range

        dongs_x, dongs_y = [], []

        # 동별로 탐색
        for dong in transaction_data['동'].unique():
            filtered_data = transaction_data[transaction_data['동'] == dong]

            # 200601부터 sequence_length(window_size)만큼 탐색
            for idx in range(len(date_range)-window_size+1): # len(date_range)-sequence_length+1
                # x,y 포함된 기간 가져오기
                current_range = date_range[:idx+window_size+1]
                
                # x 기간(y 기간 전)에 sequence_length만큼 거래가 존재하는 단지만 가져오기(단, current_range_filtered_data에는 모든 기간 포함)
                current_range_apartment_complex = filtered_data[filtered_data['계약년월'].isin(current_range[:-1])].groupby('단지').filter(lambda x: len(x) >= window_size)['단지'].unique()
                current_range_filtered_data = filtered_data[filtered_data['단지'].isin(current_range_apartment_complex)]

                # x 기간의 단지별 평단가, 시간, 경제(x 기간 중 마지막 window_size 만큼)
                current_range_filtered_x = current_range_filtered_data[current_range_filtered_data['계약년월'].isin(current_range[:-1])].groupby('단지').apply(lambda x: x.tail(window_size)).reset_index(drop=True)
                grouped_current_range_filtered_x = current_range_filtered_x.groupby('단지').agg({'제곱미터당 거래금액(만원)': list}).reset_index()['제곱미터당 거래금액(만원)'].to_list()
                grouped_current_range_filtered_time_x = current_range_filtered_x.groupby('단지').agg({'계약년월': list}).reset_index()['계약년월'].to_list()
                grouped_current_range_filtered_time_x = [[float((ts.year-pd.Timestamp('2006-01').year)*12+(ts.month-pd.Timestamp('2006-01').month)+1) for ts in sublist] for sublist in grouped_current_range_filtered_time_x]
                grouped_current_range_filtered_economy_x = [[economy_data['통화량'][ts] for ts in current_range[-1-window_size:-1]]]

                # y 기간의 단지별 평단가, 시간, 경제
                grouped_current_range_filtered_y = []
                grouped_current_range_filtered_time_y = []
                grouped_current_range_filtered_economy_y = []
                for apartment_complex in current_range_apartment_complex:
                    if current_range_filtered_data[current_range_filtered_data['단지']==apartment_complex]['계약년월'].isin([current_range[-1]]).any():                
                        grouped_current_range_filtered_y.append(current_range_filtered_data[(current_range_filtered_data['단지']==apartment_complex) & (current_range_filtered_data['계약년월']==current_range[-1])]['제곱미터당 거래금액(만원)'].to_list())
                    else:
                        grouped_current_range_filtered_y.append([0.0])
                    grouped_current_range_filtered_time_y.append([current_range[-1]])
                grouped_current_range_filtered_time_y = [[float((ts.year-pd.Timestamp('2006-01').year)*12+(ts.month-pd.Timestamp('2006-01').month)+1) for ts in sublist] for sublist in grouped_current_range_filtered_time_y]
                grouped_current_range_filtered_economy_y.append([economy_data['통화량'][current_range[-1]]])
                
                # 최대 단지 수만큼 단지별 평단가 채우기
                if len(grouped_current_range_filtered_x) < all_dong_max_apartment_complex:
                    for _ in range(all_dong_max_apartment_complex-len(grouped_current_range_filtered_x)):
                        grouped_current_range_filtered_x.append([0.0]*window_size)
                        grouped_current_range_filtered_time_x.append([0.0]*window_size)
                        grouped_current_range_filtered_y.append([0.0])
                        grouped_current_range_filtered_time_y.append([0.0])

                # y 기간의 단지별 평단가가 0이면 뒤로 빼기
                sorted_indices = np.argsort([-y[0] if y[0] != 0 else float('inf') for y in grouped_current_range_filtered_y])
                grouped_current_range_filtered_x = np.array(grouped_current_range_filtered_x)[sorted_indices].tolist()
                grouped_current_range_filtered_y = np.array(grouped_current_range_filtered_y)[sorted_indices].tolist()
                grouped_current_range_filtered_time_x = np.array(grouped_current_range_filtered_time_x)[sorted_indices].tolist()
                grouped_current_range_filtered_time_y = np.array(grouped_current_range_filtered_time_y)[sorted_indices].tolist()

                # x,y 단지별 평단가, 시간, 경제 모두 묶고 dongs에 하나씩 붙이기
                grouped_current_range_filtered_and_time_and_economy_x = []
                grouped_current_range_filtered_and_time_and_economy_x.extend([grouped_current_range_filtered_x, grouped_current_range_filtered_time_x, grouped_current_range_filtered_economy_x])
                grouped_current_range_filtered_and_time_and_economy_y = []
                grouped_current_range_filtered_and_time_and_economy_y.extend([grouped_current_range_filtered_y, grouped_current_range_filtered_time_y, grouped_current_range_filtered_economy_y])
                dongs_x.append(grouped_current_range_filtered_and_time_and_economy_x)
                dongs_y.append(grouped_current_range_filtered_and_time_and_economy_y)
        self.dongs_x = dongs_x
        self.dongs_y = dongs_y
        self.len = len(dongs_x)

    # 부동산_x, 시간_x, 경제_x, 부동산_y, 시간_y, 경제_y 
    def __getitem__(self, i):
        return torch.FloatTensor(self.dongs_x[i][0]), torch.FloatTensor(self.dongs_x[i][1]), torch.FloatTensor(self.dongs_x[i][2]), torch.FloatTensor(self.dongs_y[i][0]), torch.FloatTensor(self.dongs_y[i][1]), torch.FloatTensor(self.dongs_y[i][2])
 
    def __len__(self):
        return self.len

batch_size = 1
train_dataset = ODE_Transformer_Dataset(transaction_all, economy_all)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [15]:
for x,y,z,w,p,q in train_loader:
    print(x.shape)
    print(y.shape)
    print(z.shape)
    print(w.shape)
    print(p.shape)
    print(q.shape)
    break

torch.Size([1, 311, 5])
torch.Size([1, 311, 5])
torch.Size([1, 1, 5])
torch.Size([1, 311, 1])
torch.Size([1, 311, 1])
torch.Size([1, 1, 1])
